<a href="https://colab.research.google.com/github/diogocristovao/SPBD_tp1/blob/main/docs/labs/projs/spbd2425_tp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistemas para Processamento de Big Data
## TP2 - Energy Meter Live Monitoring


The sensor data corresponds to regular readings from 11 residential energy meters. The data covers the month of February 2024.

Each data sample has the following schema:

timestamp | sensor_id | energy
----------|-------------|-----------
timestamp | string  | float

Each energy value (KWh) corresponds to the accumulated value of the meter at the time of measurement. As such,
each meter is expected to produce a monotonically increasing series of pairs of timestamp and energy consummed up to that moment.

The meters do not start at zero or at the same value.


## Questions

For all the sensors combined:

1. For the current month and current day, compute the running total energy consumed so far. The values should be updated every 5 minutes.

2. For the current month and current day, compute the running total energy consumed so far, **as a percentage**, **compared to the same periods in February 2024**. The values should be updated every 5 minutes.

For each sensor, separately:

3. For the current month and current day, compute the running total energy consumed so far, as a percentage, **comparing the value of each individual sensor, relative to the same results for all the sensors together (as in #1)**. The values should be updated every 5 minutes. (Sorted in descending order by value and sensor.)

**Note:** For simplicity, it is fine to assume the first reading of each day can be used to start counting how much energy has been consumed so far. There is no need to interpolate/estimate the value of the meters at midnight.




## Requeriments

Solve each question using Structured Spark Streaming.

## Other Grading Criteria

+ Grading will also take into account the general clarity of the programming and of the presentation report (notebook).




### Deadline

December 6.

Penalty of 0.25 grade points per day late.

Penalty accumulates until the grade of the assignment reaches 8.0.

---
### Colab Setup


In [1]:
#@title Install PySpark
!pip install pyspark --quiet

In [2]:
#@title Download Archived February Energy Readings
!wget -q -O /tmp/readings.csv https://raw.githubusercontent.com/smduarte/spbd-2425/refs/heads/main/docs/labs/projs/energy-readings.csv
!grep "2024-02" /tmp/readings.csv > february-energy-readings.csv
!head -2 february-energy-readings.csv


2024-02-01 00:00:00;D;2615.0
2024-02-01 00:00:18;C;1098.8


In [3]:
#@title Start the Structured Source
!wget -q -O - https://github.com/smduarte/spbd-2425/raw/main/scripts/json_energy_sender.tgz  | tar xfz - 2> /dev/null

!nohup python json_energy_sender/server.py --filename json_energy_sender/energy-readings.csv --speedup 60 > /dev/null 2> /dev/null &


Note: --speedup 60, means the stream is played 60x faster than realtime. Therefore, 1 second in real time corresponds to 1 minute of stream data.


In [4]:
#@title Sample code to process the structured stream...
from pyspark.sql import *
from pyspark.sql.functions import *

spark = SparkSession \
    .builder \
    .appName("StructuredWebLogExample") \
    .getOrCreate()


# Extract a sample JSON string to infer schema
sample_json = '{"date": "2024-02-01 00:00:00", "sensor": "D", "energy": 2615.0}'
inferred_schema = schema_of_json(sample_json)


# Create DataFrame representing the stream of input
# lines from connection to logsender 7777
try:
  json_lines = spark.readStream.format("socket") \
      .option("host", "localhost") \
      .option("port", 7777) \
      .load()

  # Parse the JSON using the inferred schema
  json_lines = json_lines.withColumn("json_data", from_json(col("value"), inferred_schema)) \
    .select("json_data.*")  # Expand the JSON fields into columns


  query = json_lines \
    .writeStream \
    .outputMode("append") \
    .trigger(processingTime='1 seconds') \
    .foreachBatch(lambda df, epoch: df.show(10, False)) \
    .start()

  query.awaitTermination(19)
except Exception as err:
  print(err)
  query.stop()

+----+------+------+
|date|energy|sensor|
+----+------+------+
+----+------+------+

+-------------------+-------+------+
|date               |energy |sensor|
+-------------------+-------+------+
|2024-10-01 00:04:21|2790.18|C     |
|2024-10-01 00:04:27|5949.0 |D     |
|2024-10-01 00:04:36|2162.37|J     |
|2024-10-01 00:04:52|2682.69|I     |
|2024-10-01 00:04:24|3993.9 |H     |
|2024-10-01 00:04:33|3481.07|E     |
|2024-10-01 00:04:43|1597.49|F     |
+-------------------+-------+------+

+-------------------+-------+------+
|date               |energy |sensor|
+-------------------+-------+------+
|2024-10-01 00:14:30|2790.19|C     |
|2024-10-01 00:14:42|3481.08|E     |
|2024-10-01 00:14:48|1668.96|B     |
|2024-10-01 00:14:54|1649.25|A     |
|2024-10-01 00:15:00|2682.71|I     |
|2024-10-01 00:14:36|5949.1 |D     |
|2024-10-01 00:14:45|2162.5 |J     |
|2024-10-01 00:14:51|1597.5 |F     |
|2024-10-01 00:14:57|2080.99|G     |
+-------------------+-------+------+



Question 1

In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *

# Initialize Spark session
spark = SparkSession \
    .builder \
    .appName("EnergyMeterMonitoring") \
    .getOrCreate()

# Infer schema from a sample JSON
sample_json = '{"date": "2024-02-01 00:00:00", "sensor": "D", "energy": 2615.0}'
inferred_schema = schema_of_json(sample_json)

try:
    # Create DataFrame representing the stream of input
    json_lines = spark.readStream.format("socket") \
        .option("host", "localhost") \
        .option("port", 7777) \
        .load()

    # Debug: Check raw data
    raw_query = json_lines.writeStream \
        .outputMode("append") \
        .format("console") \
        .option("truncate", False) \
        .start()

    raw_query.awaitTermination(10)  # Run this for 10 seconds to confirm data is coming in

    # Parse the JSON data
    parsed_stream = json_lines.withColumn("json_data", from_json(col("value"), inferred_schema)) \
        .selectExpr("json_data.date as timestamp", "json_data.sensor as sensor_id", "json_data.energy as energy") \
        .withColumn("timestamp", to_timestamp(col("timestamp")))  # Convert timestamp to proper type

    # Debug: Check parsed data
    parsed_query = parsed_stream.writeStream \
        .outputMode("append") \
        .format("console") \
        .option("truncate", False) \
        .start()

    parsed_query.awaitTermination(10)  # Run this for 10 seconds to confirm parsing works

    # Calculate the running total using aggregation over all data
    running_total = parsed_stream \
        .agg(sum("energy").alias("cumulative_energy"))

    # Write the running total to the console
    query = running_total.writeStream \
        .outputMode("complete") \
        .format("console") \
        .option("truncate", False) \
        .trigger(processingTime="5 minutes")\
        .start()

    query.awaitTermination(30)  # Run for 30 seconds
except Exception as err:
    print(f"Error: {err}")
    if 'query' in locals():
        query.stop()


+-------------------+-------+------+
|date               |energy |sensor|
+-------------------+-------+------+
|2024-10-01 00:24:38|2790.2 |C     |
|2024-10-01 00:24:44|5949.1 |D     |
|2024-10-01 00:24:57|1668.98|B     |
|2024-10-01 00:25:06|2081.01|G     |
|2024-10-01 00:24:41|3993.95|H     |
|2024-10-01 00:24:51|3481.09|E     |
|2024-10-01 00:25:03|1649.26|A     |
|2024-10-01 00:25:09|2682.72|I     |
+-------------------+-------+------+

+-------------------+-------+------+
|date               |energy |sensor|
+-------------------+-------+------+
|2024-10-01 00:34:47|2790.21|C     |
|2024-10-01 00:34:53|5949.1 |D     |
|2024-10-01 00:35:03|2162.51|J     |
|2024-10-01 00:35:09|1597.51|F     |
|2024-10-01 00:34:50|3993.96|H     |
|2024-10-01 00:34:59|3481.1 |E     |
|2024-10-01 00:35:06|1668.99|B     |
+-------------------+-------+------+

+-------------------+-------+------+
|date               |energy |sensor|
+-------------------+-------+------+
|2024-10-01 00:35:12|1649.27|A     |